In [2]:
"""
EOD HD fundamentals builder — v7
Outputs the columns:
• MACD (12,26,9)   ← raw MACD
• MACD Signal      ← 9-day EMA of MACD
• ATH % Chg, Recent-High % Chg, RSI-1 yr, Fair-Value metrics, etc.
Ticker is printed without the “.US” suffix.
"""

import datetime as _dt
import requests  as _rq
import pandas    as _pd

API_TOKEN = "67ffece4b2ae08.94077168"
BASE_URL  = "https://eodhd.com/api"
HEADERS   = {"User-Agent": "EOD-Fundamentals-Client/7.0"}


def _get_json(url: str):
    r = _rq.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()


def _pct_from_high(curr, high):
    if curr is None or high in (None, 0):
        return None
    return (curr - high) / high


def _pct_to_value(curr, val):
    """(FairValue – CurrentPrice) / CurrentPrice"""
    if curr in (None, 0) or val is None:
        return None
    return (val - curr) / curr


def _strip_suffix(ticker: str) -> str:
    return ticker.rsplit(".", 1)[0]


# ---------- fundamental helpers -----------------------------------------------
def _annual_eps_dict(fund: dict) -> dict[int, float]:
    return {
        int(k[:4]): rec["epsActual"]
        for k, rec in fund.get("Earnings", {}).get("Annual", {}).items()
        if rec.get("epsActual") is not None
    }


def _sum_quarterly_eps(hist: list[dict]) -> dict[int, float]:
    eps_q = {}
    for rec in hist:
        eps = rec.get("epsActual")
        d   = _pd.to_datetime(rec.get("date") or rec.get("reportDate"), errors="coerce")
        if _pd.notna(d) and eps is not None:
            eps_q.setdefault(d.year, []).append(eps)
    return {y: sum(v) for y, v in eps_q.items() if len(v) == 4 and sum(v) != 0}


def _five_year_pe_series(df_price: _pd.DataFrame, eps_year: dict[int, float]) -> list[float]:
    pe = []
    for y in sorted(eps_year)[-5:]:
        yr_px = df_price[df_price["date"].dt.year == y]
        if not yr_px.empty and (eps := eps_year[y]) != 0:
            pe.append(yr_px.iloc[-1]["adjusted_close"] / eps)
    return pe


def CurrentPrice(ticker):
    url = f"{BASE_URL}/real-time/{ticker}?api_token={API_TOKEN}&fmt=json"
    d = _get_json(url)
    return d.get("close") or d.get("price") or d.get("lastPrice")


def ATH(ticker):
    today = _dt.date.today().strftime("%Y-%m-%d")
    url   = f"{BASE_URL}/eod/{ticker}?api_token={API_TOKEN}&from=1900-01-01&to={today}&adjusted=1&fmt=json"
    df    = _pd.DataFrame(_get_json(url))
    return None if df.empty else df["adjusted_close"].max()


# ---------- technical-indicator functions ---------------------------------------
def _latest_macd(ticker):
    """Return only the latest raw MACD value."""
    today  = _dt.date.today()
    url = (
        f"{BASE_URL}/technical/{ticker}?order=d"
        f"&from={today.replace(year=today.year-2)}&to={today}"
        f"&function=macd&fast_period=12&slow_period=26&signal_period=9"
        f"&api_token={API_TOKEN}&fmt=json"
    )
    try:
        dat = _get_json(url)
        return dat[0].get("macd") if dat else None
    except Exception:
        return None


def _latest_macd_signal(ticker):
    """Return only the latest 9-EMA of MACD (signal line)."""
    today  = _dt.date.today()
    url = (
        f"{BASE_URL}/technical/{ticker}?order=d"
        f"&from={today.replace(year=today.year-2)}&to={today}"
        f"&function=macd&fast_period=12&slow_period=26&signal_period=9"
        f"&api_token={API_TOKEN}&fmt=json"
    )
    try:
        dat = _get_json(url)
        return dat[0].get("signal") if dat else None
    except Exception:
        return None


def _latest_rsi_1y(ticker):
    today  = _dt.date.today()
    url = (
        f"{BASE_URL}/technical/{ticker}?order=d"
        f"&from={today.replace(year=today.year-2)}&to={today}"
        f"&function=rsi&period=252"
        f"&api_token={API_TOKEN}&fmt=json"
    )
    try:
        dat = _get_json(url)
        return dat[0]["rsi"] if dat else None
    except Exception:
        return None


def Fundamentals(ticker: str) -> _pd.DataFrame:
    fund = _get_json(f"{BASE_URL}/fundamentals/{ticker}?api_token={API_TOKEN}&fmt=json")

    today_dt = _dt.datetime.now()
    hist_url = (
        f"{BASE_URL}/eod/{ticker}?api_token={API_TOKEN}"
        f"&from={(today_dt.replace(year=today_dt.year-6)).strftime('%Y-%m-%d')}"
        f"&to={today_dt.strftime('%Y-%m-%d')}&adjusted=1&fmt=json"
    )
    dfp = _pd.DataFrame(_get_json(hist_url))
    if dfp.empty:
        dfp = _pd.DataFrame(columns=["date", "adjusted_close"])
    dfp["date"] = _pd.to_datetime(dfp["date"])

    # -- highs & % changes -----------------------------------------------------
    curr_price     = CurrentPrice(ticker)
    all_time_high  = ATH(ticker)
    recent_high    = dfp.loc[dfp["date"] > _pd.Timestamp(today_dt) - _pd.Timedelta(days=180),
                             "adjusted_close"].max(skipna=True)
    high_5y        = dfp["adjusted_close"].max(skipna=True)

    # -- earnings & P/E --------------------------------------------------------
    hi        = fund.get("Highlights", {})
    eps_ttm   = hi.get("DilutedEpsTTM")
    pe_api    = hi.get("PERatio")

    hist_raw  = fund.get("Earnings", {}).get("History", {})
    earnings  = list(hist_raw.values()) if isinstance(hist_raw, dict) else (hist_raw or [])
    eps_year  = _annual_eps_dict(fund) or _sum_quarterly_eps(earnings)
    pe_series = _five_year_pe_series(dfp, eps_year)
    avg_pe5y  = sum(pe_series) / len(pe_series) if len(pe_series) >= 2 else None

    if pe_api is None and eps_ttm and eps_ttm != 0:
        last_px = dfp.iloc[-1]["adjusted_close"] if not dfp.empty else curr_price
        pe_api  = last_px / eps_ttm

    eps_est   = hi.get("EPSEstimateNextYear")
    fair_val  = avg_pe5y * eps_ttm if avg_pe5y and eps_ttm else None
    fair_valF = avg_pe5y * eps_est if avg_pe5y and eps_est else None

    # -- technicals ------------------------------------------------------------
    macd_raw    = _latest_macd(ticker)
    macd_signal = _latest_macd_signal(ticker)
    rsi_1y      = _latest_rsi_1y(ticker)

    row = {
        "Ticker":             _strip_suffix(ticker),
        "Current Price":      curr_price,
        "All-Time High":      all_time_high,
        "ATH %Chg":           _pct_from_high(curr_price, all_time_high),
        "6-Month High":       recent_high,
        "Recent-High %Chg":   _pct_from_high(curr_price, recent_high),
        "5-Year High":        high_5y,
        "Current P/E":        pe_api,
        "Avg 5-Year P/E":     avg_pe5y,
        "Fair Value (TTM)":   fair_val,
        "Fair Value %Chg":    _pct_to_value(curr_price, fair_val),
        "Avg P/E × Fwd EPS":  fair_valF,
        "MACD (12,26,9)":     macd_raw,       
        "MACD Signal":        macd_signal,    
        "RSI (1 yr)":         rsi_1y,
    }
    return _pd.DataFrame([row])


def FundamentalsBuilder(tickers):
    return _pd.concat([Fundamentals(t) for t in tickers], ignore_index=True)


def SaveToCSV(df, fname):
    df.to_csv(fname, index=False)
    print(f"Saved {len(df)} rows → {fname}")


if __name__ == "__main__":
    raw = [
        "NET","CRS","ORLY","DOCN","TEM","VST","CVS","TER","RIVN","NTGR",
        "SPOT","UBER","SNOW","ACHR","ASAN","RKLB","DPZ","LYFT","NEGG",
        "MSFT","AAPL","ARM","AMZN","META","NFLX","NVDA","GOOG","RDFN",
        "MDB","Z","INTC","AMD","AFRM","COIN","TSLA","HOOD","CAVA",
        "F","GM","SMR","U","RDDT","SHOP","SOFI"
    ]
    tickers = [f"{t}.US" for t in raw]

    SaveToCSV(FundamentalsBuilder(tickers), "fundamentals_eod7.csv")


Saved 44 rows → fundamentals_eod7.csv
